# Machine Learning Attack on Arbiter PUF
## Justin Schubeck, Alex Liu, Yash Bhat
### EEE5716 - Introduction to Hardware Security and Trust: Fall 2022
---

We will first import the necessary libraries.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

First, we will read in the data from the Excel file. 

In [4]:
df = pd.read_excel('CRPSets.xls',header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,0,1,1,1,0,1,0,0,1,...,1,1,0,0,0,1,0,1,1,1
1,1,0,1,0,1,0,0,0,1,0,...,1,1,1,0,0,0,0,0,1,0
2,1,1,0,1,0,1,1,0,1,0,...,1,0,1,1,1,0,0,1,0,0
3,0,1,0,0,0,0,1,1,0,1,...,1,1,1,1,1,0,0,1,1,0
4,1,1,0,0,0,1,1,1,0,1,...,0,1,0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,1,0,1,1,1,0,0,1,0,...,1,0,0,0,0,1,1,0,0,0
11996,0,1,0,0,1,1,0,1,0,1,...,1,1,1,0,0,1,0,1,0,1
11997,1,1,1,0,0,0,1,1,0,0,...,1,1,0,1,0,1,1,0,0,0
11998,0,1,0,1,1,1,1,1,1,1,...,0,0,0,0,1,0,1,0,0,1


In [20]:
challenges = df.to_numpy()[:,:64]
responses = df.to_numpy()[:,64]

We will now split the data into a training set (80%) and a testing set (20%). This will ensure that the models are not overfitting, and the test set will be used to evaluate the model. 

In [23]:
X_train, X_test, y_train, y_test = train_test_split(challenges, 
                                                    responses,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=37) 

---
# 1. Support Vector Machine

## Training

## Test

---
# 2. Regression

## Training

## Test